# Data Merging and Cleaning

### Getting Started

Importing needed libraries and reading already gathered csv data.

In [1]:
import pandas as pd
from config import tmdb_api_key
import requests

In [2]:
numbers_budget_df = pd.read_csv('Data/tn.movie_budgets.csv')
tmdb_df = pd.read_csv('Data/tmdb_df.csv')

In [3]:
print(numbers_budget_df.shape)
print(tmdb_df.shape)

(5782, 6)
(4776, 14)


In [4]:
print(numbers_budget_df.columns)
print(tmdb_df.columns)

Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')
Index(['popularity', 'vote_count', 'video', 'poster_path', 'id', 'adult',
       'backdrop_path', 'original_language', 'original_title', 'genre_ids',
       'title', 'vote_average', 'overview', 'release_date'],
      dtype='object')


### Data Merging

We will merge on the title and year values simultaneously, as multiple movies have the same titles. We need to create these indicies to be merged on.

In [5]:
numbers_budget_df.rename(columns={'movie':'title'}, inplace=True)
numbers_budget_df['year'] = numbers_budget_df['release_date'].map(lambda x: str(x)[-4:])
tmdb_df['year'] = tmdb_df['release_date'].map(lambda x: str(x)[:4])

In [6]:
merged_df = numbers_budget_df.merge(tmdb_df, on=['title', 'year'], how='inner')
merged_df

,id_x,release_date_x,title,production_budget,domestic_gross,worldwide_gross,year,popularity,vote_count,video,poster_path,id_y,adult,backdrop_path,original_language,original_title,genre_ids,vote_average,overview,release_date_y
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009,81.649,22183,False,/6EiRUJpuoeQPghrs3YNktfnqOVh.jpg,19995,False,/bIL7ENqh1egWTxN41sM2W42JqPc.jpg,en,Avatar,"[28, 12, 14, 878]",7.5,"In the 22nd century, a paraplegic Marine is di...",2009-12-10
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,49.185,10310,False,/9zxsSzXx2UO3yNoS4j97vs4uOhc.jpg,1865,False,/bfTKq6mGHt2CzZlBU31JHF9bMog.jpg,en,Pirates of the Caribbean: On Stranger Tides,"[28, 12, 14]",6.5,Captain Jack Sparrow crosses paths with a woma...,2011-05-14
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,122.894,3997,False,/cCTJPelKGLhALq3r51A9uMonxKj.jpg,320288,False,/phxiKFDvPeQj4AbkvJLmuZEieDU.jpg,en,Dark Phoenix,"[28, 12, 878]",6.1,The X-Men face their most formidable and power...,2019-06-05
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,77.916,16653,False,/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg,99861,False,/8i6ZDk97Vyh0jHohMG4vFeFuKJh.jpg,en,Avengers: Age of Ultron,"[28, 12, 878]",7.3,When Tony Stark tries to jumpstart a dormant p...,2015-04-22
4,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",2018,240.150,19926,False,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,299536,False,/lmZFxXgJE3vgrciwuDib0N8CfQo.jpg,en,Avengers: Infinity War,"[28, 12, 878]",8.3,As the Avengers and their allies have continue...,2018-04-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,65,"Dec 31, 2007",Tin Can Man,"$12,000",$0,$0,2007,1.584,3,False,/2YMMRWzq9SzZEYau1iAtdTPN269.jpg,157185,False,NaN,en,Tin Can Man,[27],3.0,Recently dumped by his girlfirend for another ...,2007-01-01
4362,67,"Apr 28, 2006",Clean,"$10,000","$138,711","$138,711",2006,0.600,0,False,/kOk47vD8E0dScAyNVNCXhsncaE4.jpg,586483,False,NaN,en,Clean,[],0.0,A dangerous serial killer decides to go out wi...,2006-03-05
4363,71,"Aug 14, 2008",The Rise and Fall of Miss Thang,"$10,000",$401,$401,2008,0.600,0,False,/2YkIO5qjfhyICn5wXhLYMJLk152.jpg,579116,False,NaN,en,The Rise and Fall of Miss Thang,[],0.0,An irresponsible party girl begins a journey t...,2008-08-14
4364,75,"Oct 8, 2004",Primer,"$7,000","$424,760","$841,926",2004,71.750,1378,False,/xEoq2WmDzpzxhkHEsmOYOg6BPg6.jpg,14337,False,/gIFbfh9O4HBrO6numxVBxtZ9fTL.jpg,en,Primer,"[18, 878, 53]",6.9,Friends and fledgling entrepreneurs invent a d...,2004-10-08


In order to analyze genre data, we will break out the genres into a column for each genre, and assign each value a boolean. TMDB has an api function to get all the genre name-to-id relationships, which we will convert into a Python dictionary.

In [7]:
tmdb_genres = requests.get(f'https://api.themoviedb.org/3/genre/movie/list?api_key={tmdb_api_key}&language=en-US').json()['genres']
tmdb_genres

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 80, 'name': 'Crime'},
 {'id': 99, 'name': 'Documentary'},
 {'id': 18, 'name': 'Drama'},
 {'id': 10751, 'name': 'Family'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 36, 'name': 'History'},
 {'id': 27, 'name': 'Horror'},
 {'id': 10402, 'name': 'Music'},
 {'id': 9648, 'name': 'Mystery'},
 {'id': 10749, 'name': 'Romance'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 10770, 'name': 'TV Movie'},
 {'id': 53, 'name': 'Thriller'},
 {'id': 10752, 'name': 'War'},
 {'id': 37, 'name': 'Western'}]

In [8]:
genres = {}
for genre in tmdb_genres:
    genres[str(genre['id'])] = genre['name']
genres

{'28': 'Action',
 '12': 'Adventure',
 '16': 'Animation',
 '35': 'Comedy',
 '80': 'Crime',
 '99': 'Documentary',
 '18': 'Drama',
 '10751': 'Family',
 '14': 'Fantasy',
 '36': 'History',
 '27': 'Horror',
 '10402': 'Music',
 '9648': 'Mystery',
 '10749': 'Romance',
 '878': 'Science Fiction',
 '10770': 'TV Movie',
 '53': 'Thriller',
 '10752': 'War',
 '37': 'Western'}

Each genre will be looped over, creating a new column in the dataframe. Each column is mapped from the 'genre_ids' column.

In [9]:
merged_df.genre_ids

0       [28, 12, 14, 878]
1            [28, 12, 14]
2           [28, 12, 878]
3           [28, 12, 878]
4           [28, 12, 878]
              ...        
4361                 [27]
4362                   []
4363                   []
4364        [18, 878, 53]
4365             [18, 53]
Name: genre_ids, Length: 4366, dtype: object

In [10]:
for key, value in genres.items():
    merged_df[value.lower().replace(' ', '_')] = merged_df['genre_ids'].map(lambda x: key in x)

In [11]:
print(merged_df.shape)
print(merged_df.columns)

(4366, 39)
Index(['id_x', 'release_date_x', 'title', 'production_budget',
       'domestic_gross', 'worldwide_gross', 'year', 'popularity', 'vote_count',
       'video', 'poster_path', 'id_y', 'adult', 'backdrop_path',
       'original_language', 'original_title', 'genre_ids', 'vote_average',
       'overview', 'release_date_y', 'action', 'adventure', 'animation',
       'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy',
       'history', 'horror', 'music', 'mystery', 'romance', 'science_fiction',
       'tv_movie', 'thriller', 'war', 'western'],
      dtype='object')


### Data Cleaning

Movies that have no genre or revene data will not be useful in our analysis, so we will remove them. Dupliactes will also be removed. This only affects 205 data points.

In [12]:
no_genres = list(merged_df[merged_df['genre_ids'] == '[]'].index)
len(no_genres)

20

In [13]:
merged_df.drop(index=no_genres, inplace=True)

In [14]:
no_gross = list(merged_df[merged_df['worldwide_gross'] == '$0'].index)
len(no_gross)

180

In [15]:
merged_df.drop(index=no_gross, inplace=True)

In [16]:
duplicates = list(merged_df[merged_df.duplicated()].index)
len(duplicates)

5

In [17]:
merged_df.drop(index=duplicates, inplace=True)

In [18]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4161 entries, 0 to 4365
Data columns (total 39 columns):
id_x                 4161 non-null int64
release_date_x       4161 non-null object
title                4161 non-null object
production_budget    4161 non-null object
domestic_gross       4161 non-null object
worldwide_gross      4161 non-null object
year                 4161 non-null object
popularity           4161 non-null float64
vote_count           4161 non-null int64
video                4161 non-null bool
poster_path          4145 non-null object
id_y                 4161 non-null int64
adult                4161 non-null bool
backdrop_path        3979 non-null object
original_language    4161 non-null object
original_title       4161 non-null object
genre_ids            4161 non-null object
vote_average         4161 non-null float64
overview             4161 non-null object
release_date_y       4161 non-null object
action               4161 non-null bool
adventure         

Many of the columns are unecessary, and some have unhelpful data types. We will also create a few new columns from the existing that may be useful, and rename ones with unhelpful titles.

In [19]:
merged_df.drop(columns = ['id_x', 'release_date_x', 'id_y', 'poster_path', 'adult', 'backdrop_path', 'video'], inplace=True)

In [20]:
merged_df['production_budget'] = merged_df['production_budget'].map(lambda x: int(str(x).strip('$').replace(',', '')))
merged_df['domestic_gross'] = merged_df['domestic_gross'].map(lambda x: int(str(x).strip('$').replace(',', '')))
merged_df['worldwide_gross'] = merged_df['worldwide_gross'].map(lambda x: int(str(x).strip('$').replace(',', '')))

In [21]:
merged_df['worldwide_profit(loss)'] = merged_df['worldwide_gross'] - merged_df['production_budget']

In [22]:
merged_df['roi'] = merged_df['worldwide_profit(loss)'] * 100 / merged_df['production_budget']

In [23]:
merged_df['year'] = merged_df['year'].map(int)
merged_df['month'] = merged_df['release_date_y'].map(lambda x: int(x.split('-')[1]))
merged_df['day'] = merged_df['release_date_y'].map(lambda x: int(x.split('-')[2]))

In [24]:
merged_df.rename(columns = {'release_date_y': 'release_date'}, inplace = True)

In [25]:
merged_df.columns

Index(['title', 'production_budget', 'domestic_gross', 'worldwide_gross',
       'year', 'popularity', 'vote_count', 'original_language',
       'original_title', 'genre_ids', 'vote_average', 'overview',
       'release_date', 'action', 'adventure', 'animation', 'comedy', 'crime',
       'documentary', 'drama', 'family', 'fantasy', 'history', 'horror',
       'music', 'mystery', 'romance', 'science_fiction', 'tv_movie',
       'thriller', 'war', 'western', 'worldwide_profit(loss)', 'roi', 'month',
       'day'],
      dtype='object')

Rearanging the columns makes the data easier to read.

In [26]:
cleaned_df = merged_df[['title','original_title', 'year', 'month', 'day', 'release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                           'worldwide_profit(loss)', 'roi', 'popularity', 'vote_count', 'vote_average', 'original_language', 'genre_ids',
        'action', 'adventure', 'animation', 'comedy', 'crime',
       'documentary', 'drama', 'family', 'fantasy', 'history', 'horror',
       'music', 'mystery', 'romance', 'science_fiction', 'tv_movie',
       'thriller', 'war', 'western', 'overview']]

In [27]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4161 entries, 0 to 4365
Data columns (total 36 columns):
title                     4161 non-null object
original_title            4161 non-null object
year                      4161 non-null int64
month                     4161 non-null int64
day                       4161 non-null int64
release_date              4161 non-null object
production_budget         4161 non-null int64
domestic_gross            4161 non-null int64
worldwide_gross           4161 non-null int64
worldwide_profit(loss)    4161 non-null int64
roi                       4161 non-null float64
popularity                4161 non-null float64
vote_count                4161 non-null int64
vote_average              4161 non-null float64
original_language         4161 non-null object
genre_ids                 4161 non-null object
action                    4161 non-null bool
adventure                 4161 non-null bool
animation                 4161 non-null bool
comedy     

In [28]:
cleaned_df

,title,original_title,year,month,day,release_date,production_budget,domestic_gross,worldwide_gross,worldwide_profit(loss),...,horror,music,mystery,romance,science_fiction,tv_movie,thriller,war,western,overview
0,Avatar,Avatar,2009,12,10,2009-12-10,425000000,760507625,2776345279,2351345279,...,False,False,False,False,True,False,False,False,False,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011,5,14,2011-05-14,410600000,241063875,1045663875,635063875,...,False,False,False,False,False,False,False,False,False,Captain Jack Sparrow crosses paths with a woma...
2,Dark Phoenix,Dark Phoenix,2019,6,5,2019-06-05,350000000,42762350,149762350,-200237650,...,False,False,False,False,True,False,False,False,False,The X-Men face their most formidable and power...
3,Avengers: Age of Ultron,Avengers: Age of Ultron,2015,4,22,2015-04-22,330600000,459005868,1403013963,1072413963,...,False,False,False,False,True,False,False,False,False,When Tony Stark tries to jumpstart a dormant p...
4,Avengers: Infinity War,Avengers: Infinity War,2018,4,25,2018-04-25,300000000,678815482,2048134200,1748134200,...,False,False,False,False,True,False,False,False,False,As the Avengers and their allies have continue...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4353,In the Company of Men,In the Company of Men,1997,3,28,1997-03-28,25000,2883661,2883661,2858661,...,False,False,False,False,False,False,False,False,False,Two business executives--one an avowed misogyn...
4354,Exeter,Exeter,2015,1,16,2015-01-16,25000,0,489792,464792,...,True,False,False,False,False,False,True,False,False,"During an all-night, drug-fueled party at an a..."
4356,Slacker,Slacker,1991,7,5,1991-07-05,23000,1227508,1227508,1204508,...,False,False,False,False,False,False,False,False,False,"Austin, Texas, is an Eden for the young and un..."
4364,Primer,Primer,2004,10,8,2004-10-08,7000,424760,841926,834926,...,False,False,False,False,True,False,True,False,False,Friends and fledgling entrepreneurs invent a d...


Saving the cleaned and merged dataframe to a csv file.

In [29]:
cleaned_df.to_csv('Data/cleaned_data.csv', index=False)